In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout

In [5]:
data = []
labels = []
classes = 411
cur_path = os.path.join(os.getcwd(), 'CASIA-Iris-Lamp')

for i in range(1, classes+1):
    subdir = str(i).zfill(3)  # zero-padding the subdirectory number
    for side in ['L', 'R']:
        path = os.path.join(cur_path, subdir, side)
        images = os.listdir(path)
        
        for a in images:
            try:
                image = Image.open(os.path.join(path, a))
                image = image.resize((30,30))
                image = np.array(image)
                data.append(image)
                labels.append(i)
            except:
                print("Error loading image")
                
data = np.array(data)
labels = np.array(labels)

Error loading image
Error loading image
Error loading image


In [6]:
data

array([[[ 47,  54,  58, ...,  53,  35,  26],
        [ 49,  53,  64, ...,  57,  43,  31],
        [ 55,  61,  64, ...,  73,  51,  38],
        ...,
        [146, 149, 152, ..., 105,  98,  86],
        [142, 146, 149, ..., 105,  97,  85],
        [135, 140, 142, ..., 102,  94,  82]],

       [[ 52,  57,  61, ...,  73,  56,  37],
        [ 57,  66,  76, ...,  80,  61,  47],
        [ 62,  71,  80, ...,  89,  79,  58],
        ...,
        [158, 161, 164, ..., 111, 107,  97],
        [153, 157, 159, ..., 113, 108,  96],
        [144, 149, 152, ..., 113, 106,  94]],

       [[ 59,  60,  72, ...,  79,  65,  52],
        [ 69,  71,  77, ...,  84,  69,  57],
        [ 76,  82,  90, ..., 103,  82,  64],
        ...,
        [138, 141, 143, ..., 124, 121, 115],
        [134, 139, 140, ..., 127, 122, 113],
        [131, 135, 138, ..., 124, 120, 111]],

       ...,

       [[ 22,  25,  29, ...,  85,  75,  64],
        [ 22,  25,  30, ...,  89,  80,  72],
        [ 22,  25,  32, ...,  96,  87,  78

In [7]:
labels

array([  1,   1,   1, ..., 411, 411, 411])

In [8]:
print(data.shape,labels.shape)
X_train,X_test,y_train,y_test=train_test_split(data,labels,test_size=0.2,random_state=42)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
y_train = to_categorical(y_train,412)
y_test = to_categorical(y_test,412)

(16212, 30, 30) (16212,)
(12969, 30, 30) (3243, 30, 30) (12969,) (3243,)


In [9]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(412, activation='softmax'))


In [10]:
#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [24]:
epochs = 10
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))
model.save("my_model1.h5")

Epoch 1/10
406/406 [==============================] - 29s 72ms/step - loss: 0.6164 - accuracy: 0.8184 - val_loss: 0.2974 - val_accuracy: 0.9334
Epoch 2/10
406/406 [==============================] - 29s 72ms/step - loss: 0.6246 - accuracy: 0.8146 - val_loss: 0.3190 - val_accuracy: 0.9269
Epoch 3/10
406/406 [==============================] - 30s 73ms/step - loss: 0.6351 - accuracy: 0.8112 - val_loss: 0.3762 - val_accuracy: 0.9137
Epoch 4/10
406/406 [==============================] - 50s 123ms/step - loss: 0.6534 - accuracy: 0.8084 - val_loss: 0.3293 - val_accuracy: 0.9309
Epoch 5/10
406/406 [==============================] - 31s 77ms/step - loss: 0.6245 - accuracy: 0.8193 - val_loss: 0.2961 - val_accuracy: 0.9346
Epoch 6/10
406/406 [==============================] - 41s 101ms/step - loss: 0.6008 - accuracy: 0.8244 - val_loss: 0.3768 - val_accuracy: 0.9140
Epoch 7/10
406/406 [==============================] - 40s 98ms/step - loss: 0.6129 - accuracy: 0.8201 - val_loss: 0.3053 - val_accurac

In [25]:
# Predict classes of test images
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Evaluate accuracy on test set
_, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', test_acc)

Test accuracy: 0.9031760692596436


In [2]:
from PIL import Image
import numpy as np
import tensorflow as tf
model = tf.keras.models.load_model('my_model1.h5')
test_image = Image.open('CASIA-Iris-Lamp/111/L/S2111L01.jpg').convert('L')
test_image = test_image.resize((30, 30))
test_t = np.array(test_image)
test_t = np.expand_dims(test_t, axis=-1)
test_t = np.expand_dims(test_t, axis=0)

pred = np.argmax(model.predict(test_t), axis=-1)
print(pred)

[111]
